In [4]:
from __future__ import division
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from PIL import Image
import cv2
sys.path.append("..")

In [5]:
from utils import label_map_util
from utils import visualization_utils as vis_util

In [6]:
# Selecting model.
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES=90

In [7]:
#Load a (frozen) Tensorflow model into memory.
detection_graph=tf.Graph()
with detection_graph.as_default():
    od_graph_def=tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
    
label_map=label_map_util.load_labelmap(PATH_TO_LABELS)
categories=label_map_util.convert_label_map_to_categories(label_map,max_num_classes=NUM_CLASSES,use_display_name=True)
category_index=label_map_util.create_category_index(categories)

In [8]:
def count_nonblack(img):
    return img.any(axis=-1).sum()

In [9]:
def team_name(image, show=False):
    # define the list of boundaries
    boundaries = [
    ([17, 15, 100], [50, 56, 200]), #red
    ([25, 146, 190], [96, 174, 250]) #yellow
    ]
    i = 0
    for (lower, upper) in boundaries:
        # create NumPy arrays from the boundaries
        lower = np.array(lower, dtype = "uint8")
        upper = np.array(upper, dtype = "uint8")

        # find the colors within the specified boundaries and apply
        # the mask
        mask = cv2.inRange(image, lower, upper)
        output = cv2.bitwise_and(image, image, mask = mask)
        tot_pix = count_nonblack(image)
        color_pix = count_nonblack(output)
        ratio = color_pix/tot_pix
#         print("ratio is:", ratio)
        if ratio > 0.01 and i == 0:
            return 'red'
        elif ratio > 0.01 and i == 1:
            return 'yellow'

        i += 1
        
        if show == True:
            cv2.imshow("images", np.hstack([image, output]))
            if cv2.waitKey(0) & 0xFF == ord('q'):
              cv2.destroyAllWindows() 
    return 'not_sure'

In [10]:
image=cv2.imread('1234.png')
resize =cv2.resize(image,(640,360))
team_name(resize,show=True)

'yellow'

In [12]:
#working with video

cap=cv2.VideoCapture('vid.mp4')
fourcc=cv2.VideoWriter_fourcc(*'XVID')
out=cv2.VideoWriter('op.avi',fourcc,20.0,(640,480))

#Running Tensorflow Session

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        count=0
        while(True):
            ret,img_np=cap.read()
            count+=1
            if ret:
                h=img_np.shape[0]
                w=img_np.shape[1]
            if not ret:
                break
            if count%1==0:
                image_expanded=np.expand_dims(img_np,axis=0)
                image_tensor=detection_graph.get_tensor_by_name('image_tensor:0')
                boxes=detection_graph.get_tensor_by_name('detection_boxes:0')
                scores=detection_graph.get_tensor_by_name('detection_scores:0')
                classes=detection_graph.get_tensor_by_name('detection_classes:0')
                num_detections=detection_graph.get_tensor_by_name('num_detections:0')
                (boxes, scores, classes, num_detections) = sess.run(
                      [boxes, scores, classes, num_detections],
                      feed_dict={image_tensor: image_expanded})
                
                vis_util.visualize_boxes_and_labels_on_image_array(
                                          img_np,
                                          np.squeeze(boxes),
                                          np.squeeze(classes).astype(np.int32),
                                          np.squeeze(scores),
                                          category_index,
                                          use_normalized_coordinates=True,
                                          line_thickness=3,
                                          min_score_thresh=0.6)
                frame_number=count
                
                #dictionary to team and there cor-ordinates
                loc={}
                for n in range(len(scores[0])):
                    if scores[0][n]>0.60:
                        ymin = int(boxes[0][n][0] * h)
                        xmin = int(boxes[0][n][1] * w)
                        ymax = int(boxes[0][n][2] * h)
                        xmax = int(boxes[0][n][3] * w)   
                        
                        for cat in categories:
                            if cat['id']==classes[0][n]:
                                label=cat['name']
                                
                        if label=='person':
                            crop_im=img_np[ymin:ymax,xmin:xmax]
                            color = team_name(crop_im)
                            if color!='not_sure':
                                coords=(xmin,ymin)
                                if color=='red':
                                    loc[coords]='PERU'
                                else:
                                    loc[coords]='AUS'
                                    
                                    
                for key in loc.keys():
                    text_pos=str(loc[key])
                    cv2.putText(img_np, text_pos, (key[0], key[1]-20), cv2.FONT_HERSHEY_SIMPLEX, 0.50, (255, 0, 0), 2)
                    
            cv2.imshow('img',img_np)
            out.write(img_np)
            
            if cv2.waitKey(10) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
                cap.release()
                break
            
        